# Database management

> Classes and functions for managing access to the databases

In [1]:
#| default_exp db_mgmt

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export

from kedro.config import OmegaConfigLoader
from kedro.framework.project import settings
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm import tqdm

import psycopg2



[01/28/25 09:07:15] INFO     Using                                                                  ]8;id=472258;file:///Users/moritzbeckmail.de/miniconda3/envs/if_sdk/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=3524;file:///Users/moritzbeckmail.de/miniconda3/envs/if_sdk/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/moritzbeckmail.de/miniconda3/envs/if_sdk/lib/python3.11/site-p                
                             ackages/kedro/framework/project/rich_logging.yml' as logging                          
                             configuration.                                                                        

In [4]:
#| export

def get_db_credentials():
    
    """
    Fetch PostgreSQL database credentials from the configuration file of the kedro project.

    Uses `OmegaConfigLoader` to load credentials stored under `credentials.postgres`.

    Returns:
        dict: A dictionary with the database connection details (e.g., host, port, user, password, dbname).
    """

    conf_path = str(Path(settings.CONF_SOURCE))
    conf_loader = OmegaConfigLoader(conf_source=conf_path)
    db_credentials = conf_loader["credentials"]["postgres"]

    return db_credentials

In [5]:
#| export

class SQLDatabase:
    """
    A class to represent a SQL database.

    Attributes:
    ----------
    credentials : dict
        A dictionary containing the database connection credentials.
    connection : psycopg2.connection
        The database connection object.

    Methods:
    -------
    connect():
        Connects to the database using the provided credentials.
    
    close():
        Closes the database connection.
    
    execute_query(query: str):
        Executes the given SQL query.
    
    execute_multiple_queries(queries: list, params: list = None, fetchrows: bool = False):
        Executes a list or iterable of SQL queries. 
        If fetchrows is True, iterates over queries and fetches rows.
        If fetchrows is False, uses executemany for batch execution.
    """

    def __init__(self):
        """
        Initializes the SQLDatabase object with the provided database connection credentials.
        """
        self._credentials = get_db_credentials()["con"]
        self.connection = None

    def connect(self):
        """
        Connects to the database using the provided credentials.
        """
        if not self.connection:
            self.connection = psycopg2.connect(self._credentials)

    def close(self):
        """
        Closes the database connection.
        """
        if self.connection:
            self.connection.close()
            self.connection = None

    def execute_query(self, query: str, params: tuple = None, fetchall: bool = False, fetchone = False):
        """
        Executes the given SQL query with optional parameters.
        
        Parameters:
        -----------
        query : str
            The SQL query to be executed.
        
        params : tuple, optional
            The parameters to be passed to the query. Defaults to None.

        fetchall : bool, optional
            Whether to fetch all rows from the query result. Defaults to False.

        fetchone : bool, optional
            Whether to fetch only one row from the query result. Defaults to False.
        
        Returns:
        --------
        result : list
            A list of tuples representing the query result rows.
        """

        # Check if not both fetchall and fetchone are True
        if fetchall and fetchone:
            raise ValueError("Both fetchall and fetchone cannot be True")

        if not self.connection:
            self.connect()

        with self.connection.cursor() as cur:
            cur.execute(query, params)
            if fetchall:
                result = cur.fetchall()
            elif fetchone:
                result = cur.fetchone()
            else:
                result = None

        self.connection.commit()
        return result
    

    def execute_multiple_queries(self, queries: list | str, params: list = None, fetchrows: bool = False):
        """
        Executes a list or iterable of SQL queries. 
        If fetchrows is True, iterates over queries and fetches rows.
        If fetchrows is False, tries to use executemany for batch execution.

        Parameters:
        -----------
        queries : list or str
            A list of SQL queries to be executed, or a single query as a string.
        
        params : list, optional
            A list of tuples containing parameters for each query. Defaults to None.
        
        fetchrows : bool, optional
            Whether to fetch rows from the queries. Defaults to False (use executemany).
        
        Returns:
        --------
        results : list
            A list of results for each executed query, or None if using executemany.
        """
        if not self.connection:
            self.connect()

        results = []
        with self.connection.cursor() as cur:
            if fetchrows:

                if isinstance(queries, str):
                    # Convert single query to a list of same length as params
                    queries = [queries] * len(params)

                # Iterate over queries and fetch rows
                for idx, query in enumerate(queries):
                    query_params = params[idx] if params else None
                    cur.execute(query, query_params)
                    result = cur.fetchone()
                    results.append(result)  # Collect the result of each query
            else:

                if not isinstance(queries, str):
                    # In this case only one query with multiple params can be executed (raise error)
                    raise ValueError("Multiple queries with multiple params are not supported when using executemany. Set fetchrows=True")
                                
                cur.executemany(queries, params)

        self.connection.commit()

        return results if fetchrows else None

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()

